In [1]:
import os
import numpy as np
import cv2
import fnmatch
from matplotlib import pyplot as plt
from pathlib  import Path
import h5py


In [13]:
#input_patch = '/home/jupyter/src/CLAM_annotation/PreRT_S_wsi_Patches_Seg/patches'
input_preds ='/home/jupyter/project/WSI/tcga_stroma_prediction'
dir_til ='SB_til'
dir_tumor ='SB_tumor'
dir_stroma ='SB_stroma'

In [14]:
def immune_subtype_clam(input_path, tile_size=256, h5_files =None):
    
    files  = fnmatch.filter(os.listdir(f"{input_path}/{dir_til}/"), '*.png') #contour h5y files by create_patches_fn.py from pickles by /home/jupyter/src/Mask_annotation_jang.ipynb

    with open('result_Tumor_tils_density_SB.csv', 'w') as outfile :
        outfile.write('file_name,immune_infl,immune_excl,immune_des, til, tumor, stroma, til_tumor, til_stroma, tumor_stroma\n')
        for slide_name in files :
            filename = slide_name#.replace('.svs', '_gray.png')
            
            tumor = input_path + f'/{dir_tumor}/' + filename
            til = input_path + f'/{dir_til}/' + filename            
            stroma = input_path + f'/{dir_stroma}/' + filename

            if not (os.path.exists(tumor) and os.path.exists(til) and os.path.exists(stroma)) : continue

            tumor_img_gray = cv2.imread(tumor, cv2.IMREAD_GRAYSCALE)
            til_img_gray = cv2.imread(til, cv2.IMREAD_GRAYSCALE)
            stroma_img_gray = cv2.imread(stroma, cv2.IMREAD_GRAYSCALE)


            # cv2.imwrite(f'./test_wsi_til/overlay/tumor.png',tumor_img_gray)



            # tissue_mask_img_gray = np.zeros_like(tumor_img_gray)
            # tile_coords = h5py.File(os.path.join(input_patch, slide_name))['coords']            
            # for coord in tile_coords:
            #     j = int(coord[0]/tile_size)
            #     i = int(coord[1]/tile_size)
            #     tissue_mask_img_gray[i,j]=1
            


            #plt.imshow(til_img)
            #plt.show()
            #print('til:',til_img.shape)

            # cv2.imwrite(f'./test_wsi_til/overlay/til.png',til_img_gray)
            til_img_gray[til_img_gray < 128] = 0
            til_img_gray[til_img_gray >= 128] = 1

            # cv2.imwrite(f'./test_wsi_til/overlay/stroma.png',stroma_img_gray)
            stroma_img_gray[stroma_img_gray < 128] = 0
            stroma_img_gray[stroma_img_gray >= 128] = 1
            #print('stroma:',stroma_img.shape)
            #plt.imshow(stroma_img)
            #plt.show()            

            tumor_img_gray = cv2.resize(tumor_img_gray, (til_img_gray.shape[1],til_img_gray.shape[0]))
            tumor_img_gray[tumor_img_gray < 128] = 0
            tumor_img_gray[tumor_img_gray >= 128] = 1

            out = np.zeros((tumor_img_gray.shape[0], tumor_img_gray.shape[1], 3))

            out[:,:,2] = tumor_img_gray #R
            out[:,:,1] = stroma_img_gray #G
            out[:,:,0] = til_img_gray #B

            cv2.imwrite(f'{input_preds}/overlay/tumor.png',tumor_img_gray*255)
            cv2.imwrite(f'{input_preds}/overlay/stroma.png',stroma_img_gray*255)
            cv2.imwrite(f'{input_preds}/overlay/til.png',til_img_gray*255)

            s =filename.replace('.png','_comb.png')
            cv2.imwrite(f'{input_preds}/overlay/{s}',out*255)

            tissue_mask_img_gray = cv2.bitwise_or(til_img_gray, tumor_img_gray)
            tissue_mask_img_gray = cv2.bitwise_or(tissue_mask_img_gray, stroma_img_gray)
            s= filename.replace('.png','_mask.png')
            cv2.imwrite(f'{input_preds}/overlay/{s}',tissue_mask_img_gray*255)


            #immune inf. : TIL & Tumor
            inf = np.zeros_like(out)
            inf[:,:,2] = tumor_img_gray #R
            inf[:,:,1] = np.zeros_like(stroma_img_gray) #G
            inf[:,:,0] = til_img_gray #B
            mask = cv2.bitwise_and(tumor_img_gray, til_img_gray)

            s=filename.replace('.png','_inf.png')
            cv2.imwrite(f'{input_preds}/overlay/{s}',cv2.bitwise_and(inf, inf, mask= mask)*255)
            
            #immune des. : Stroma & Tumor
            des = np.zeros_like(out)
            des[:,:,2] = tumor_img_gray #R
            des[:,:,1] = stroma_img_gray #G
            des[:,:,0] = np.zeros_like(til_img_gray) #B
            s= filename.replace('.png','_des.png')

            mask = cv2.bitwise_and(tumor_img_gray, stroma_img_gray)
            cv2.imwrite(f'{input_preds}/overlay/{s}',cv2.bitwise_and(des, des, mask= mask)*255)

            #immune exc. : TIL & Stroma
            exc = np.zeros_like(out)
            exc[:,:,2] = np.zeros_like(tumor_img_gray) #R
            exc[:,:,1] = stroma_img_gray #G
            exc[:,:,0] = til_img_gray #B

            mask = cv2.bitwise_and(til_img_gray, stroma_img_gray)
            s= filename.replace('.png','_exc.png')
            cv2.imwrite(f'{input_preds}/overlay/{s}',cv2.bitwise_and(exc, exc, mask= mask)*255)

            



            overlap_tt = til_img_gray + tumor_img_gray #overlap TIL and Tumor
            overlap_ts=til_img_gray + stroma_img_gray#overlap of til and stroma
            overlap_tst=tissue_mask_img_gray + til_img_gray
            
            c_overlap_tt = overlap_tt > 1 #binarize
            c_overlap_ts=overlap_ts > 1
            c_overlap_tst=overlap_tst > 1
            
            imm_inf=0
            imm_excl=0
            imm_des=0
            c_tissue= 0
           
            i=0


            #xr = float(slide.properties['openslide.mpp-x'])  # pixel resolution at x direction
            #yr = float(Slide.properties['openslide.mpp-y'])  # pixel resolution at y direction
            # generate X, Y coordinates for tiling
            #M, N = [round(112 / xr), round(112 / yr)] =424
            #xr,yr=0.264169
            #dim=(97262,63858)
            M,N = 17,17
            tiles = [tissue_mask_img_gray[x:x+M,y:y+N] for x in range(0,tissue_mask_img_gray.shape[0],M) for y in range(0,tissue_mask_img_gray.shape[1],N)]
            for x in range(0,tissue_mask_img_gray.shape[0],M):
                for y in range(0,tissue_mask_img_gray.shape[1],N):
                    #print(i)
                    #print(tiles[i].sum())
                    if (tiles[i].sum())>0:
                        tissue_sum=tiles[i].sum()
                        c_tissue+=1 #count the number of tissue tiles
                        overlap_count_tt = sum(map(lambda x : x == True, c_overlap_tt[x:x+M,y:y+N]))#count the overlap ratio
                        overlap_count_ts = sum(map(lambda x : x == True, c_overlap_ts[x:x+M,y:y+N]))
                        overlap_count_tst = sum(map(lambda x : x == True, c_overlap_tst[x:x+M,y:y+N]))
                        sum_overlap_tt = np.sum(overlap_count_tt)
                        sum_overlap_ts = np.sum(overlap_count_ts)
                        sum_overlap_tst = np.sum(overlap_count_tst)
                        #print(sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]),sum_overlap_ts/np.count_nonzero(stroma_img[x:x+M,y:y+N]),sum_overlap_tst)
                        if np.isnan(sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]))==False and sum_overlap_tt/np.count_nonzero(tumor_img_gray[x:x+M,y:y+N]) > 0.4:
                            imm_inf+=1
                        elif np.isnan(sum_overlap_ts/np.count_nonzero(stroma_img_gray[x:x+M,y:y+N]))==False and sum_overlap_ts/np.count_nonzero(stroma_img_gray[x:x+M,y:y+N]) >0.2:
                            imm_excl+=1
                        else:
                            imm_des+=1
                    i+=1
            print(filename,imm_inf,imm_excl,c_tissue,imm_des)
            if c_tissue == 0 : continue
            outfile.write(filename
                              + ',' + str(imm_inf/c_tissue)
                              + ',' + str(imm_excl/c_tissue)
                              + ',' + str(imm_des/c_tissue) 
                              + ',' + str(til_img_gray.sum())
                              + ',' + str(tumor_img_gray.sum())
                              + ',' + str(stroma_img_gray.sum())
                              + ',' + str((tumor_img_gray & til_img_gray).sum())
                              + ',' + str((til_img_gray & stroma_img_gray).sum())
                              + ',' + str((tumor_img_gray & stroma_img_gray).sum())
                              + '\n')
    outfile.close()
#input_path = "/home/ext_jang_inyeop_mayo_edu/hwangt-share/post_process/VanAbel_Oropharynx_Tumor_TIL_Stroma/Van Abel-K_req32469_OPX TILs HE/"
immune_subtype_clam(input_preds)



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:146: RuntimeWarning: invalid value encountered in long_scalars
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars


1_S_42235235.png 0 0 192 192
0_S_47313628.png 2 1 270 267
1_S_48743044.png 0 0 315 315
0_S_48144111.png 7 0 280 273
1_S_39695576.png 1 2 157 154
2_S_44253804.png 0 0 262 262
0_B_1180276.png 0 0 152 152
2_B_1579721.png 0 0 193 193
3_B_1578202.png 0 0 257 257
2_S_47040045.png 2 0 313 311
3_B_1310565.png 0 0 289 289
2_S_44146184.png 0 0 366 366
0_S_46317432.png 2 0 202 200
1_S_44266420.png 1 0 94 93
2_S_38529621.png 0 0 145 145
2_S_46887366.png 1 1 313 311
1_S_41846012.png 0 0 228 228
1_S_37870225.png 3 0 184 181
1_S_43649604.png 0 0 305 305
2_S_44247203.png 0 0 212 212
0_S_48692634.png 13 0 498 485
2_S_44023072.png 0 0 163 163
2_S_41148996.png 3 0 333 330
0_S_46484673.png 0 0 149 149
2_S_41693294.png 1 0 493 492
1_S_45069824.png 1 0 280 279
2_S_45185788.png 0 0 91 91
2_S_51889744.png 1 0 384 383
2_S_37522913.png 0 0 271 271
1_S_38589971.png 0 1 217 216
2_S_39334321.png 0 0 161 161
2_S_41758184.png 9 0 536 527
2_S_45813665.png 0 0 446 446
2_S_44341198.png 11 0 129 118
3_S_49193844.png 0 0